#**BELLUNO 100mx100m**

In [ ]:
#sessionInfo()

In [ ]:
#library()

In [22]:
install.packages(c('spatstat', 'SpatEntropy', 'tibble', 'dplyr','janitor','sp','PROJ', 'plyr'), 
                repos = "http://cran.rstudio.com/")

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [2]:
library(spatstat)
library(SpatEntropy)
library(tibble)
library(plyr) 
library(dplyr)
library(janitor)
library(sp)
library(PROJ)

Loading required package: spatstat.data

Loading required package: nlme

Loading required package: rpart


spatstat 1.64-1       (nickname: ‘Help you I can, yes!’) 
For an introduction to spatstat, type ‘beginner’ 



Attaching package: ‘dplyr’


The following objects are masked from ‘package:plyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize


The following object is masked from ‘package:nlme’:

    collapse


The following objects are masked from ‘package:rgeos’:

    intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘janitor’


The following objects are masked from ‘package:stats’:

    chisq.test, fisher.test




In [25]:
# install gdal
system('sudo apt-get update')
system('sudo apt-get install libgdal-dev libproj-dev')

# install required R packages
install.packages(c("rgdal","rgeos","maptools"), repos='http://cran.rstudio.com/')

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [1]:
library(rgdal)
library(rgeos)
library(maptools)

Loading required package: sp

rgdal: version: 1.5-12, (SVN revision 1018)
Geospatial Data Abstraction Library extensions to R successfully loaded
Loaded GDAL runtime: GDAL 2.2.3, released 2017/11/20
Path to GDAL shared files: /usr/share/gdal/2.2
GDAL binary built with GEOS: TRUE 
Loaded PROJ runtime: Rel. 4.9.3, 15 August 2016, [PJ_VERSION: 493]
Path to PROJ shared files: (autodetected)
Linking to sp version:1.4-2

rgeos version: 0.5-3, (SVN revision 634)
 GEOS runtime version: 3.6.2-CAPI-1.10.2 
 Linking to sp version: 1.4-2 
 Polygon checking: TRUE 


Checking rgeos availability: TRUE



# **DATA**

---



In [3]:
source('/content/Functions.R')

In [4]:
dataBL = read.csv("/content/dataBL_100C_all.csv")

In [27]:
dataBL = read.csv("/content/data_length.csv")

In [28]:
head(dataBL)

,id,Ritagliato_build_area,Ritagliato_build_pc,COMUNE,LENGTH,COUNT
,<int>,<dbl>,<dbl>,<fct>,<dbl>,<int>
1,204145,2402.69357,24.0269362,Feltre,195.72131,2
2,204144,152.36111,1.5236111,Feltre,38.75886,1
3,204143,0.00000,0.0000000,Feltre,0.00000,0
4,204142,358.90396,3.5890395,Feltre,0.00000,0
5,204141,0.00000,0.0000000,Feltre,0.00000,0
6,204140,45.46146,0.4546146,Feltre,101.71571,1


In [5]:
dataBL$COMUNE = as.character(dataBL$COMUNE)
dataBL$COMUNE = ifelse(dataBL$COMUNE == 'ArsiÃ¨','Arsiè', dataBL$COMUNE)
dataBL$COMUNE = ifelse(dataBL$COMUNE == 'San NicolÃ² di Comelico', 'San Nicolò di Comelico', dataBL$COMUNE)
dataBL$COMUNE = ifelse(dataBL$COMUNE == 'ZoppÃ¨ di Cadore', 'Zoppè di Cadore', dataBL$COMUNE)
dim(dataBL)
head(dataBL)

[1] 738068     12

,id,left,top,right,bottom,Ritagliato,Ritaglia_1,COD_PROV,PRO_COM,COMUNE,fid,DNZone
,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<chr>,<int>,<int>
1,204145,4472600,2547400,4472700,2547300,2402.69357,24.0269362,25,25021,Feltre,4972,242
2,204144,4472600,2547500,4472700,2547400,152.36111,1.5236111,25,25021,Feltre,4931,311
3,204143,4472600,2547600,4472700,2547500,0.00000,0.0000000,25,25021,Feltre,4931,311
4,204142,4472600,2547700,4472700,2547600,358.90396,3.5890395,25,25021,Feltre,4869,231
5,204141,4472600,2547800,4472700,2547700,0.00000,0.0000000,25,25021,Feltre,4869,231
6,204140,4472600,2547900,4472700,2547800,45.46146,0.4546146,25,25021,Feltre,4869,231


In [6]:
dataBL$build = ifelse(dataBL$DNZone <200,1,0)
dataBL$build = ifelse(dataBL$Ritaglia_1 > 5, 1, dataBL$build)
head(dataBL)

,id,left,top,right,bottom,Ritagliato,Ritaglia_1,COD_PROV,PRO_COM,COMUNE,fid,DNZone,build
,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<chr>,<int>,<int>,<dbl>
1,204145,4472600,2547400,4472700,2547300,2402.69357,24.0269362,25,25021,Feltre,4972,242,1
2,204144,4472600,2547500,4472700,2547400,152.36111,1.5236111,25,25021,Feltre,4931,311,0
3,204143,4472600,2547600,4472700,2547500,0.00000,0.0000000,25,25021,Feltre,4931,311,0
4,204142,4472600,2547700,4472700,2547600,358.90396,3.5890395,25,25021,Feltre,4869,231,0
5,204141,4472600,2547800,4472700,2547700,0.00000,0.0000000,25,25021,Feltre,4869,231,0
6,204140,4472600,2547900,4472700,2547800,45.46146,0.4546146,25,25021,Feltre,4869,231,0


In [7]:
dataBL = dataBL[,c(1,10,13)]
head(dataBL)

,id,COMUNE,build
,<int>,<chr>,<dbl>
1,204145,Feltre,1
2,204144,Feltre,0
3,204143,Feltre,0
4,204142,Feltre,0
5,204141,Feltre,0
6,204140,Feltre,0


In [8]:
comuni = unique(dataBL$COMUNE[order(dataBL$COMUNE)])[-1]
print(comuni)

 [1] "Agordo"                       "Alano di Piave"              
 [3] "Alleghe"                      "Alpago"                      
 [5] "Arsiè"                        "Auronzo di Cadore"           
 [7] "Belluno"                      "Borca di Cadore"             
 [9] "Borgo Valbelluna"             "Calalzo di Cadore"           
[11] "Canale d'Agordo"              "Cencenighe Agordino"         
[13] "Cesiomaggiore"                "Chies d'Alpago"              
[15] "Cibiana di Cadore"            "Colle Santa Lucia"           
[17] "Comelico Superiore"           "Cortina d'Ampezzo"           
[19] "Danta di Cadore"              "Domegge di Cadore"           
[21] "Falcade"                      "Feltre"                      
[23] "Fonzaso"                      "Gosaldo"                     
[25] "La Valle Agordina"            "Lamon"                       
[27] "Limana"                       "Livinallongo del Col di Lana"
[29] "Longarone"                    "Lorenzago di Cadore"     

### TABLE ###

## **Consumption Analysis - Sprawl**

In [ ]:
shp_comuni = readOGR(dsn = '/content', layer = 'data18_BL_100', use_iconv = TRUE, dropNULLGeometries = FALSE,
encoding="UTF-8")
shp_comuni@data[['COMUNE']] = as.character(shp_comuni@data[['COMUNE']])
dim(shp_comuni)

OGR data source with driver: ESRI Shapefile 
Source: "/content", layer: "data18_BL_100"
with 738068 features
It has 7 fields


[1] 738068      7

In [ ]:
shp_comuni@data = left_join(shp_comuni@data, dataBL[,c(1,2)], by = 'id')
dim(shp_comuni)

[1] 764530      8

In [ ]:
shp_comuni@data[['COMUNE']] = ifelse(shp_comuni@data[['COMUNE']] == 'ArsiÃ¨','Arsiè', shp_comuni@data[['COMUNE']])
shp_comuni@data[['COMUNE']] = ifelse(shp_comuni@data[['COMUNE']] == 'San NicolÃ² di Comelico', 'San Nicolò di Comelico', shp_comuni@data[['COMUNE']])
shp_comuni@data[['COMUNE']] = ifelse(shp_comuni@data[['COMUNE']] == 'ZoppÃ¨ di Cadore', 'Zoppè di Cadore', shp_comuni@data[['COMUNE']])
shp_comuni@data[['COMUNE']] = ifelse(shp_comuni@data[['COMUNE']] == "Cortina d\'Ampezzo", paste("Cortina d'Ampezzo", shp_comuni@data[['COMUNE']])
shp_comuni@data[['COMUNE']] = ifelse(shp_comuni@data[['COMUNE']] == "Chies d\'Alpago", "Chies d'Alpago", shp_comuni@data[['COMUNE']])
shp_comuni@data[['COMUNE']] = ifelse(shp_comuni@data[['COMUNE']] == "Canale d\'Agordo", "Canale d'Agordo", shp_comuni@data[['COMUNE']])
dim(shp_comuni)

[1] 738068      7

In [ ]:
unique(shp_comuni@data[['COMUNE']])[1:61]

[1] "Feltre"                       "San NicolÃ² di Comelico"     
 [3] "Cortina d'Ampezzo"            "Colle Santa Lucia"           
 [5] "Comelico Superiore"           "Rocca Pietore"               
 [7] "San Gregorio nelle Alpi"      "Cibiana di Cadore"           
 [9] "Longarone"                    "Cesiomaggiore"               
[11] "Chies d'Alpago"               "Sedico"                      
[13] "Canale d'Agordo"              "Auronzo di Cadore"           
[15] "San Vito di Cadore"           "Livinallongo del Col di Lana"
[17] "Gosaldo"                      "San Tomaso Agordino"         
[19] "Limana"                       "Pieve di Cadore"             
[21] "Calalzo di Cadore"            "Cencenighe Agordino"         
[23] "ArsiÃ¨"                       "Ponte nelle Alpi"            
[25] "Santo Stefano di Cadore"      "Falcade"                     
[27] "Rivamonte Agordino"           "Seren del Grappa"            
[29] "La Valle Agordina"            "Domegge di Cadore"           
[31] "Alano di Piave"               "Alleghe"                     
[33] "Belluno"                      "San Pietro di Cadore"        
[35] "Santa Giustina"               "Fonzaso"                     
[37] "Lorenzago di Cadore"          "Lozzo di Cadore"             
[39] "Sospirolo"                    "Soverzene"                   
[41] "Sovramonte"                   "ZoppÃ¨ di Cadore"            
[43] "Quero Vas"                    "Lamon"                       
[45] "Voltago Agordino"             "Borca di Cadore"             
[47] "Vallada Agordina"             "Valle di Cadore"             
[49] "Pedavena"                     "Perarolo di Cadore"          
[51] "Vigo di Cadore"               "Vodo Cadore"                 
[53] "Selva di Cadore"              "Taibon Agordino"             
[55] "Tambre"                       "Ospitale di Cadore"          
[57] "Danta di Cadore"              "Agordo"                      
[59] "Alpago"                       "Val di Zoldo"                
[61] "Borgo Valbelluna"

In [ ]:
for(n in 1:1){
  grid_comune = subset(shp_comuni, COMUNE == comuni[n])

  jpeg(filename = paste('/content/plots/Belluno_', comuni[n],'_100m.jpg', sep = ''), 
       width = 700, height = 700, quality = 100, res = 120)
  
  plot(grid_comune, col = grid_comune@data$frag, main = comuni[n],
       sub = 'Pixel lattice with 100x100m resolution of soil fragmentation \n due to roofed built-up areas (Black) and liner infrastructures (Red)')
  
  dev.off()

}

In [ ]:
n=1
grid_comune = subset(shp_comuni, COMUNE == comuni[n])

plot(grid_comune, col = grid_comune@data$frag, main = comuni[n],
       sub = 'Pixel lattice with 100x100m resolution of soil fragmentation \n due to roofed built-up areas (Black) and liner infrastructures (Red)')
  

In [ ]:
Encoding(shp_comuni@data[['COMUNE']]) <- "UTF-8"

# **LOOP**

---



In [96]:
rows = 904
cols = 802 # 725008

gc()

for(i in c(3:5,7,8,10:17,19:61)){

  print(c(i, comuni[i]))

  # Data Matrix
  grid = dataBL
  grid$build[grid$COMUNE != comuni[i]] = NA
  grid = grid[order(grid[,'id'], grid[,'build']),]
  grid = grid[!duplicated(grid$id),]

  grid = matrix(grid$build, nrow = rows, ncol = cols, byrow = FALSE)
  grid = remove_empty(grid, which = c("rows", "cols"), quiet = TRUE)

  # Shannon Z
  shZ = shannonZ_IR(grid, breaks = 1000)
  gc()
  #write.csv(do.call("rbind", list(unlist(shZ))), file = paste('/content/shannZ/shZBL_', comuni[i],'.csv', sep = ''))

  # Observation Window
  #grid_comune = readOGR(dsn = '/content', layer = 'shape_red')
  #grid_comune@data = left_join(grid_comune@data, dataBL, by = c('id'='id','COMUNE'='COMUNE' ))
  #grid_comune@data$frag = ifelse(grid_comune@data$LENGTH>50, 2, 0)
  #grid_comune@data$frag = ifelse(grid_comune@data$build ==1, 1, grid_comune@data$frag)

  grid_comune = subset(shp_comuni, COMUNE == comuni[i])
  grid_comune@data = as.data.frame(grid_comune@data[,5])
  names(grid_comune@data) = 'build'
  gc()
  win = as.owin(grid_comune)

 # Centroids
  cc = gCentroid(grid_comune, byid = TRUE)
  coord = as.data.frame(cc@coords)
  ccP = ppp(coord$x, coord$y, win)
  gc()

  # Distance Matrix
  dmat = round(pairdist(ccP), digits = 0)
  dmat[lower.tri(dmat, diag = TRUE)] = NA 

  # Altieri Entropy
  maxdist = sqrt(diff(win$xrange)^2+diff(win$yrange)^2)
  distbreaks = c(0, 100, 200, 401, 601, 1001, 15001, 2001, maxdist)
  
  altieri = spat_entropy_IR(grid, dmat, distbreaks, shZ, breaks = 1000)
  write.csv(do.call("rbind", list(unlist(altieri))), file = paste('/content/altieri/AltieriBL_', comuni[i],'.csv', sep = ''))

  local.sum = altieri$res.local + altieri$mut.local
  gc()

  # Plots
  #jpeg(filename = paste('/content/plots/Belluno_', comuni[i],'_100m.jpg', sep = ''), 
  #     width = 700, height = 700, quality = 100, res = 120)
  
 # plot(grid_comune, col = grid_comune@data$build, main = comuni[i],
 #      sub = 'Pixel lattice with 100x100m resolution of roofed built-up areas (Black)')
  
 # dev.off()
  
  jpeg(filename = paste('/content/plots/BellunoEntropy_', comuni[i],'_100m.jpg', sep = ''), 
       width = 1000, height = 700, quality = 100, res = 120)

  barplot(height = rbind(altieri$mut.local/local.sum, altieri$res.local/local.sum),
          beside = FALSE,
          col = c('darkgrey', 'white'),
          names.arg = c('w1','w2',' w3', 'w4', 'w5', 'w6', 'w7', 'w8'),
          main = comuni[i],
          sub = 'Partial Information (Gray) and Partial Residual Entropy (White) 
        in proportional terms for each distance range')
  
  dev.off()
  gc()
}

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,28438048,1518.8,82490518,4405.5,93739862,5006.3
Vcells,62356684,475.8,130749560,997.6,130749560,997.6


[1] "3"       "Alleghe"
[1] "Building Z variable..."
[1] "Computing entropy..."
[1] "DONE"


ERROR: ignored

### SAVE OUTPUT FOLDERS ###

In [ ]:
files2zip = dir('/content/altieri', full.names = TRUE)
zip(zipfile = '/content/ZipAltieri_BL', files = files2zip)

files2zip = dir('/content/plots', full.names = TRUE)
zip(zipfile = '/content/ZipPlots_BL', files = files2zip)

files2zip = dir('/content/shannZ', full.names = TRUE)
zip(zipfile = '/content/ZipShannZ_BL', files = files2zip)

# **TEST**

---



### Altieri Entropy

In [ ]:
for(n in c(18)){
  grid = dataBL
  grid$frag[grid$COMUNE != comuni[n]] = NA
  grid = grid[order(grid[,'id'], grid[,'frag']),]
  grid = grid[!duplicated(grid$id),]

  grid = matrix(grid$frag, nrow = rows, ncol = cols, byrow = FALSE)
  grid = remove_empty(grid, which = c("rows", "cols"), quiet = TRUE)

  shZ = shannonZ_IR(grid, breaks = 1000)
  write.csv(do.call("rbind", list(unlist(shZ))), file = paste('/content/shannZ/shZBL.f_', comuni[n],'.csv', sep = ''))
}

#write.csv(do.call("rbind", list(unlist(shZ))), file = paste('/content/shannZ/shZBL.f_', comuni[n],'.csv', sep = ''))


In [21]:
for(n in c(7)){

  # Data Matrix
  grid = dataBL
  grid$build[grid$COMUNE != comuni[n]] = NA
  grid = grid[order(grid[,'id'], grid[,'build']),]
  grid = grid[!duplicated(grid$id),]

  grid = matrix(grid$build, nrow = rows, ncol = cols, byrow = FALSE)
  grid = remove_empty(grid, which = c("rows", "cols"), quiet = TRUE)

  # Shannon Z
  shZ = read.csv(paste("/content/shn/shZBL_",comuni[n],".csv", sep = ''))
  shZ

  # Observation Window
  grid_comune = subset(shp_comuni, COMUNE == comuni[n])
  #shp_comuni = readOGR(dsn = '/content', layer = 'grid_auronzo_100')
  head(grid_comune@data)
  dim(grid_comune@data)

  grid_comune@data = as.data.frame(grid_comune@data[,5])

  names(grid_comune@data) = 'build'
  head(grid_comune@data)
  gc()

  win = as.owin(grid_comune)

  # Centroids
  cc = gCentroid(grid_comune, byid = TRUE)
  coord = as.data.frame(cc@coords)
  ccP = ppp(coord$x, coord$y, win)

  # Distance Matrix
  dmat = round(pairdist(ccP, squared = F), digits = 0)
  dmat[lower.tri(dmat, diag = TRUE)] = NA 

  # Altieri Entropy
  maxdist = sqrt(diff(win$xrange)^2+diff(win$yrange)^2)
  distbreaks = c(0, 100, 201, 401, 601, 1001, 1501, 2001, maxdist)

  altieri = spat_entropy_IR2(grid, dmat, distbreaks, shannZ = shZ, breaks = 1000)
  write.csv(do.call("rbind", list(unlist(altieri))), file = paste('/content/altieri/AltieriBL_', comuni[n],'.csv', sep = ''))

  local.sum = altieri$res.local + altieri$mut.local

 jpeg(filename = paste('/content/plots/BellunoEntropy_', comuni[n],'_100m.jpg', sep = ''), 
       width = 1000, height = 700, quality = 100, res = 120)

  barplot(height = rbind(altieri$mut.local/local.sum, altieri$res.local/local.sum),
          beside = FALSE,
          col = c('darkgrey', 'white'),
          names.arg = c('w1','w2',' w3', 'w4', 'w5', 'w6', 'w7','w8'),
          main = comuni[n],
          sub = 'Partial Information (Gray) and Partial Residual Entropy (White) 
        in proportional terms for each distance range')
  
  dev.off()
}

# PARTS

In [12]:
rows = 904
cols = 802 # 725008

In [13]:
n=6

In [14]:
# Data Matrix
dataBL$build[dataBL$COMUNE != comuni[n]] = NA
dataBL = dataBL[order(dataBL[,'id'], dataBL[,'build']),]
dataBL = dataBL[!duplicated(dataBL$id),]

dataBL = matrix(dataBL$build, nrow = rows, ncol = cols, byrow = FALSE)
dataBL = remove_empty(dataBL, which = c("rows", "cols"), quiet = TRUE)

In [15]:
dim(dataBL)

# Shannon Z
shZ = read.csv(paste("/content/shn/shZBL_",comuni[n],".csv", sep = ''))
shZ

[1] 154 246

X,probabilities.couple1,probabilities.couple2,probabilities.couple3,probabilities.abs.frequency1,probabilities.abs.frequency2,probabilities.abs.frequency3,probabilities.proportion1,probabilities.proportion2,probabilities.proportion3,shannon.Z
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,0,1,11,246830871,7687774,59685,0.9695675,0.03019807,0.0002344465,0.1376168


In [20]:
gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,3042209,162.5,55735362,2976.6,91220234,4871.7
Vcells,5147925,39.3,64108190,489.2,100169046,764.3


In [9]:
shp_comuni = readOGR(dsn = '/content', layer = 'shape_red')

OGR data source with driver: ESRI Shapefile 
Source: "/content", layer: "shape_red"
with 738068 features
It has 4 fields


In [12]:
head(shp_comuni@data)

,id,Ritagliato,COMUNE,LENGTH
,<dbl>,<dbl>,<fct>,<dbl>
0,204145,2402.69357,Feltre,195.72131
1,204144,152.36111,Feltre,38.75886
2,204143,0.00000,Feltre,0.00000
3,204142,358.90396,Feltre,0.00000
4,204141,0.00000,Feltre,0.00000
5,204140,45.46146,Feltre,101.71571


In [10]:
shp_comuni@data$COMUNE = as.character(shp_comuni@data$COMUNE)
shp_comuni@data$COMUNE = ifelse(shp_comuni@data$COMUNE == 'ArsiÃ¨',
                                'Arsiè', shp_comuni@data$COMUNE)
shp_comuni@data$COMUNE = ifelse(shp_comuni@data$COMUNE == 'San NicolÃ² di Comelico', 
                                'San Nicolò di Comelico', shp_comuni@data$COMUNE)
shp_comuni@data$COMUNE = ifelse(shp_comuni@data$COMUNE == 'ZoppÃ¨ di Cadore', 
                                'Zoppè di Cadore', shp_comuni@data$COMUNE)

In [14]:
unique(shp_comuni@data$COMUNE)[1:61]

[1] "Feltre"                       "San Nicolò di Comelico"      
 [3] "Cortina d'Ampezzo"            "Colle Santa Lucia"           
 [5] "Comelico Superiore"           "Rocca Pietore"               
 [7] "San Gregorio nelle Alpi"      "Cibiana di Cadore"           
 [9] "Longarone"                    "Cesiomaggiore"               
[11] "Chies d'Alpago"               "Sedico"                      
[13] "Canale d'Agordo"              "Auronzo di Cadore"           
[15] "San Vito di Cadore"           "Livinallongo del Col di Lana"
[17] "Gosaldo"                      "San Tomaso Agordino"         
[19] "Limana"                       "Pieve di Cadore"             
[21] "Calalzo di Cadore"            "Cencenighe Agordino"         
[23] "Arsiè"                        "Ponte nelle Alpi"            
[25] "Santo Stefano di Cadore"      "Falcade"                     
[27] "Rivamonte Agordino"           "Seren del Grappa"            
[29] "La Valle Agordina"            "Domegge di Cadore"           
[31] "Alano di Piave"               "Alleghe"                     
[33] "Belluno"                      "San Pietro di Cadore"        
[35] "Santa Giustina"               "Fonzaso"                     
[37] "Lorenzago di Cadore"          "Lozzo di Cadore"             
[39] "Sospirolo"                    "Soverzene"                   
[41] "Sovramonte"                   "Zoppè di Cadore"             
[43] "Quero Vas"                    "Lamon"                       
[45] "Voltago Agordino"             "Borca di Cadore"             
[47] "Vallada Agordina"             "Valle di Cadore"             
[49] "Pedavena"                     "Perarolo di Cadore"          
[51] "Vigo di Cadore"               "Vodo Cadore"                 
[53] "Selva di Cadore"              "Taibon Agordino"             
[55] "Tambre"                       "Ospitale di Cadore"          
[57] "Danta di Cadore"              "Agordo"                      
[59] "Alpago"                       "Val di Zoldo"                
[61] "Borgo Valbelluna"

In [11]:
shp_comuni@data = left_join(shp_comuni@data, dataBL, by = c('id'='id','COMUNE'='COMUNE' ))
dim(shp_comuni@data)

[1] 738068      5

In [17]:
head(shp_comuni@data)

,id,Ritagliato,COMUNE,LENGTH,build
,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
1,204145,2402.69357,Feltre,195.72131,1
2,204144,152.36111,Feltre,38.75886,0
3,204143,0.00000,Feltre,0.00000,0
4,204142,358.90396,Feltre,0.00000,0
5,204141,0.00000,Feltre,0.00000,0
6,204140,45.46146,Feltre,101.71571,0


In [16]:
# Observation Window
shp_comuni = subset(shp_comuni, COMUNE == comuni[n])
#shp_comuni = readOGR(dsn = '/content', layer = 'grid_auronzo_100')
head(shp_comuni@data)
dim(shp_comuni@data)

,id,Ritagliato,COMUNE,LENGTH,build
,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
102572,569682,0,Auronzo di Cadore,46.40066,0
102573,569681,0,Auronzo di Cadore,0.00000,0
102574,569680,0,Auronzo di Cadore,0.00000,0
102575,569679,0,Auronzo di Cadore,0.00000,0
102576,569678,0,Auronzo di Cadore,0.00000,0
102577,569677,0,Auronzo di Cadore,0.00000,0


[1] 22565     5

In [17]:
shp_comuni@data = as.data.frame(shp_comuni@data[,5])

names(shp_comuni@data) = 'build'
head(shp_comuni@data)
gc()

,build
,<dbl>
1,0
2,0
3,0
4,0
5,0
6,0


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,3040544,162.4,70556807,3768.2,98956532,5284.9
Vcells,5145722,39.3,86032586,656.4,107540732,820.5


In [18]:
win = as.owin(shp_comuni)

#plot(grid_comune, col = grid_comune@data$build, main = comuni[n],
      # sub = 'Pixel lattice with 100x100m resolution of roofed built-up areas (Black)')

In [19]:
gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,3160594,168.8,56445446,3014.6,98956532,5284.9
Vcells,5452089,41.6,68826069,525.2,107540732,820.5


In [20]:
# Centroids
cc = gCentroid(shp_comuni, byid = TRUE)
coord = as.data.frame(cc@coords)
ccP = ppp(coord$x, coord$y, win)

In [21]:
ccP$n

[1] 22565

In [49]:
gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2986246,159.5,41327840,2207.2,93044815,4969.2
Vcells,5917184,45.2,43615130,332.8,133087687,1015.4


In [ ]:
# Distance Matrix
dmat = round(pairdist(ccP, squared = F), digits = 0)
dmat[lower.tri(dmat, diag = TRUE)] = NA 

In [ ]:
dim(dmat)

In [ ]:
# Altieri Entropy
maxdist = sqrt(diff(win$xrange)^2+diff(win$yrange)^2)
distbreaks = c(0, 100, 201, 401, 601, 1001, 1501, 2001, maxdist)

In [ ]:
altieri = spat_entropy_IR2(dataBL, dmat, distbreaks, shannZ = shZ, breaks = 1000)
altieri

In [ ]:
write.csv(do.call("rbind", list(unlist(altieri))), file = paste('/content/altieri/AltieriBL_', comuni[n],'.csv', sep = ''))

In [ ]:
#altieri = spat_entropy_IR(grid, adjlist, breaks = 80, shZ)
#altieri
local.sum = altieri$res.local + altieri$mut.local

In [ ]:
barplot(height = rbind(altieri$mut.local/local.sum, altieri$res.local/local.sum),
          beside = FALSE,
          col = c('darkgrey', 'white'),
          names.arg = c('w1','w2',' w3', 'w4', 'w5', 'w6', 'w7','w8'),
          main = comuni[n],
          sub = 'Partial Information (Gray) and Partial Residual Entropy (White) 
        in proportional terms for each distance range')

In [61]:
  jpeg(filename = paste('/content/plots/BellunoEntropy_', comuni[n],'_100m.jpg', sep = ''), 
       width = 1000, height = 700, quality = 100, res = 120)

  barplot(height = rbind(altieri$mut.local/local.sum, altieri$res.local/local.sum),
          beside = FALSE,
          col = c('darkgrey', 'white'),
          names.arg = c('w1','w2',' w3', 'w4', 'w5', 'w6', 'w7','w8'),
          main = comuni[n],
          sub = 'Partial Information (Gray) and Partial Residual Entropy (White) 
        in proportional terms for each distance range')
  
  dev.off()

png 
  2

In [ ]:
##ingredients:
  #1) Z marginal frequencies
  P.zr = shZ$probabilities$proportion
  names(P.zr) = shZ$probabilities$couple
  P.zr

00           01           11 
0.9754237723 0.0244238934 0.0001523343

In [ ]:
 #2) W marginal frequencies and Z|wk conditional frequencies
  n.dist = length(distbreaks)-1
  QQ = sum(shZ$probabilities$abs.frequency)
  P.zr.cond.wk = vector("list", n.dist)
  P.wk = numeric(n.dist)
  datavec = c(grid[!is.na(grid)])

In [ ]:
length(datavec)

[1] 22565

In [ ]:
gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,3220912,172.1,5727207,305.9,5727207,305.9
Vcells,516736478,3942.4,927684779,7077.7,898805153,6857.4


In [ ]:
dd = 1
adj.mat = matrix(0, length(datavec), length(datavec))

In [ ]:
indx = as.matrix(which(dmat > distbreaks[dd] & dmat <= distbreaks[dd+1], arr.ind = T))

In [ ]:
#### PROBLEMA ####

 for (dd in 1:n.dist)
  {
    adj.mat = matrix(0, length(datavec), length(datavec))
    indx = as.matrix(which(dmat > distbreaks[dd] & dmat <= distbreaks[dd+1], arr.ind = T))
    for(j in 1:nrow(indx)) adj.mat[indx[[j,1]], indx[[j,2]]] = 1

    output = pair_count_IR(datavec, adj.mat, breaks = 10000, missing.cat=NULL)
    P.zr.cond.wk[[dd]] = output$probabilities$proportion
    names(P.zr.cond.wk[[dd]]) = output$probabilities$couple
    P.wk[dd] = output$Qk/QQ

    adj.mat = NULL
    indx = NULL
    gc()
  }

In [ ]:
### SKIP ###
##partial terms
  res.local = mut.local = numeric(n.dist)
  for(dd in 1:n.dist)
  {
    cond.probs = as.numeric(P.zr.cond.wk[[dd]][P.zr.cond.wk[[dd]] > 0])
    marg.probs = as.numeric(P.zr[P.zr.cond.wk[[dd]] > 0])
    res.local[dd] = sum(cond.probs*log(1/cond.probs))
    mut.local[dd] = sum(cond.probs*log(cond.probs/marg.probs))
  }
  res.global = sum(P.wk*res.local)
  mut.global = sum(P.wk*mut.local)
  
  ##output
  altieri = list(mut.global = mut.global, res.global = res.global, shannZ = shZ$shannon.Z,
              mut.local = mut.local, res.local = res.local,
              pwk = P.wk, pzr.marg = P.zr, pzr.cond = P.zr.cond.wk,
              Q = QQ, Qk = P.wk*QQ)

# CODE FRAGMENTATION

In [ ]:
gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2245400,120.0,5510364,294.3,4287471,229.0
Vcells,6118902,46.7,14786712,112.9,14593860,111.4


In [ ]:
dataBL_f = read.csv("/content/dataBL.csv")
dim(dataBL_f)
head(dataBL_f)

[1] 738068      4

,id,Ritagliato,COMUNE,LENGTH
,<dbl>,<dbl>,<fct>,<dbl>
1,204145,2402.69357,Feltre,195.72131
2,204144,152.36111,Feltre,38.75886
3,204143,0.00000,Feltre,0.00000
4,204142,358.90396,Feltre,0.00000
5,204141,0.00000,Feltre,0.00000
6,204140,45.46146,Feltre,101.71571


In [ ]:
dataBL_f$frag = ifelse(dataBL_f$LENGTH>10,2,0)
dataBL_f$frag = ifelse(dataBL_f$Ritagliato>200, 1, dataBL_f$frag)
dataBL_f$COMUNE = as.character(dataBL_f$COMUNE)
#dataBL_f$id = as.integer(dataBL_f$id)
dim(dataBL_f)
head(dataBL_f)

[1] 738068      5

,id,Ritagliato,COMUNE,LENGTH,frag
,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
1,204145,2402.69357,Feltre,195.72131,1
2,204144,152.36111,Feltre,38.75886,2
3,204143,0.00000,Feltre,0.00000,0
4,204142,358.90396,Feltre,0.00000,1
5,204141,0.00000,Feltre,0.00000,0
6,204140,45.46146,Feltre,101.71571,2


In [ ]:
dataBL_f = left_join(dataBL, dataBL_f[,c(1,5)], by = 'id')
dim(dataBL_f)
head(dataBL_f)

[1] 795109      5

,X,id,COMUNE,build,frag
,<int>,<dbl>,<chr>,<int>,<dbl>
1,1,204145,Feltre,1,1
2,2,204144,Feltre,0,2
3,3,204143,Feltre,0,0
4,4,204142,Feltre,0,1
5,5,204141,Feltre,0,0
6,6,204140,Feltre,0,2


In [ ]:
n = 1

rows = 904
cols = 802 # 725008

# Data Matrix
grid = dataBL_f
grid$frag[grid$COMUNE != comuni[n]] = NA
grid = grid[order(grid[,'id'], grid[,'frag']),]
grid = grid[!duplicated(grid$id),]

dim(grid)

[1] 725008      5

In [ ]:
grid = matrix(grid$frag, nrow = rows, ncol = cols, byrow = FALSE)
grid = remove_empty(grid, which = c("rows", "cols"), quiet = TRUE)

dim(grid)

[1] 89 68

In [ ]:
# Shannon Z
shZ = shannonZ_IR(grid, breaks = 20)
shZ
write.csv(do.call("rbind", list(unlist(shZ))), file = paste('/content/shannZ/shZBL.f_', comuni[n],'.csv', sep = ''))

[1] "Building Z variable..."
[1] "Computing entropy..."
[1] "DONE"


Registered S3 method overwritten by 'cli':
  method     from    
  print.boxx spatstat



couple,abs.frequency,proportion
<fct>,<dbl>,<dbl>
00,1412040,0.43243939
01,472361,0.14466127
02,998514,0.30579642
11,39340,0.01204793
12,166914,0.05111766
22,176121,0.05393732


In [ ]:
rows = 904
cols = 802 # 725008

for(n in 18:length(comuni)){
  # Data Matrix
  grid = dataBL_f
  grid$frag[grid$COMUNE != comuni[n]] = NA
  grid = grid[order(grid[,'id'], grid[,'frag']),]
  grid = grid[!duplicated(grid$id),]

  grid = matrix(grid$frag, nrow = rows, ncol = cols, byrow = FALSE)
  grid = remove_empty(grid, which = c("rows", "cols"), quiet = TRUE)

  shZ = shannonZ_IR(grid, breaks = 20)
  write.csv(do.call("rbind", list(unlist(shZ))), file = paste('/content/shannZ/shZBL.f_', comuni[n],'.csv', sep = ''))
  gc()

}

In [55]:
gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2987898,159.6,10833847,578.6,93044815,4969.2
Vcells,300081056,2289.5,1004872654,7666.6,1256090817,9583.3


In [54]:
spat_entropy_IR2 = function(data, dmat, dist.breaks, shannZ, breaks, missing.cat=NULL)
{

#shannZ =  read.csv(paste("Belluno/Results/ShannonZ_frag/shZBL_",comuni[i],".csv", sep = ''))


  ##ingredients:
  #1) Z marginal frequencies
  P.zr = shannZ[8:10]
  names(P.zr) = c('00','01','11')
  P.zr
  
  #2) W marginal frequencies and Z|wk conditional frequencies
  n.dist = length(dist.breaks)-1
  QQ = sum(shannZ[5:7])
  P.zr.cond.wk = vector("list", n.dist)
  P.wk = numeric(n.dist)
  datavec = c(data[!is.na(data)])
  for (dd in 1:n.dist)
  {
    adj.mat = matrix(NA, length(datavec), length(datavec))
    indx = as.matrix(which(dmat > distbreaks[dd] & dmat <= distbreaks[dd+1], arr.ind = T))
    for(j in 1:nrow(indx)) adj.mat[indx[[j,1]], indx[[j,2]]] = 1
    
    output = pair_count_IR(datavec, adj.mat, breaks = breaks, missing.cat)
    P.zr.cond.wk[[dd]] = output$probabilities$proportion
    names(P.zr.cond.wk[[dd]]) = output$probabilities$couple
    P.wk[dd] = output$Qk/QQ
    
    adj.mat = NULL
    indx = NULL
    gc()
  }
  
  ##partial terms
  res.local = mut.local = numeric(n.dist)
  for(dd in 1:n.dist)
  {
    cond.probs = as.numeric(P.zr.cond.wk[[dd]][P.zr.cond.wk[[dd]] > 0])
    marg.probs = as.numeric(P.zr[P.zr.cond.wk[[dd]] > 0])
    res.local[dd] = sum(cond.probs*log(1/cond.probs))
    mut.local[dd] = sum(cond.probs*log(cond.probs/marg.probs))
  }
  res.global = sum(P.wk*res.local)
  mut.global = sum(P.wk*mut.local)
  
  ##output
  return(list(mut.global = mut.global, res.global = res.global, shannZ = shannZ$shannon.Z,
              mut.local = mut.local, res.local = res.local,
              pwk = P.wk, pzr.marg = P.zr, pzr.cond = P.zr.cond.wk,
              Q = QQ, Qk = P.wk*QQ))
}
